In [23]:
import math
import numpy as np

row = 10 
column = 10 
def Load_data(file_name):
    g_w = []
    t_l = []
    n_d = []
    with open(file_name) as f:
        file=f.readlines()
        gw=file[2:12]
        tl=file[16:20]
        nd=file[24:]
        for a in gw:
            ad=a.strip('\n').split(' ')
            g_w1=[]
            for a1 in ad:
                a1=int(a1)
                g_w1.append(a1)
            g_w.append(g_w1)
        for b in tl:
            t_l.append([int(b.strip('\n').split(' ')[2]),int(b.strip('\n').split(' ')[3])])
        for c in nd:
            n_d1=[]
            cd=c.strip('\n').split(' ')
            while '' in cd:
                 cd.remove('')
            for c1 in cd: 
                c1=float(c1)
                n_d1.append(c1)
            n_d.append(n_d1)
    return g_w, t_l, n_d
  
    
def Probability(x, y, grid_world):#相邻的两个cell，在 cell 1 情况下走到 cell 2 的概率
    o = 0
    if((y-1 >=0) and (grid_world[x][y-1]==1)):
        o += 1
    elif ((y+1 < column) and (grid_world[x][y+1]==1)):
        o += 1         
    elif ((x-1 >=0) and (grid_world[x-1][y]==1)):
        o += 1
    elif ((x+1 < row) and (grid_world[x+1][y]==1)):
        o += 1
    else: 
        pass            
    if (o > 0):
        return 1/o    
    else:
        return 0  

def Distance(towerlocation, grid_world):
    number_precistion = 1
    distance_matrix = []
    observations = []  
    max_distance = 0#最大实际距离
    min_distance = 0 #最小噪声距离
    max_distance = 0 #最大噪声距离
    unit = 0.1**number_precistion
    tower_x, tower_y = 0, 0 #塔的坐标
    row, column = np.array(grid_world).shape
    max_distance = math.sqrt((row-1)**2 + (column-1)**2)
    min_noisy = 0.7 
    max_noisy = 1.3
    for i in range(0, int(max_distance * max_noisy/unit)+1):
        observations.extend([round(i*unit,number_precistion)])
    
    for tower in towerlocation:
        prob_matrix = []
        for cell in STATES:
            tower_x = tower[0]
            tower_y = tower[1]
            distance = math.sqrt((AvailableCOORDINATION[cell][0]-tower_x)**2+(AvailableCOORDINATION[cell][1]-tower_y)**2)
            min_distance = round(min_noisy * distance, number_precistion)
            max_distance = round(max_noisy * distance, number_precistion)
            probability = 1/((max_distance - min_distance + unit)/unit)
            prob_matrix.append([])
            for i in observations:
                if (min_distance <= i and i <= max_distance):
                    prob_matrix[cell-1].extend([probability])
                else:
                    prob_matrix[cell-1].extend([0])                    
        distance_matrix.append(prob_matrix)
    return distance_matrix, observations


def CPT(grid_world):
    states = []
    transition_matrix = []
    initial_probability = []
    total_states = 0
    valid_states = 0
    x = 0
    y = 0
    upx = 0
    upy = 0
    validcoord = {}    
    for xx in grid_world: 
        for yy in xx: 
            if yy == 1:
                validcoord[(valid_states+1)] =[x, y]
                states.extend([valid_states+1])
                valid_states += 1 
                total_states += 1
                y += 1
            else:
                total_states +=1
                y += 1
        y = 0
        x += 1
    init_p = 1/valid_states
    initial_probability = np.ones((1, valid_states))*init_p
    transition_matrix = np.zeros((valid_states, valid_states))    
    for xx in range (valid_states):
        for yy in range (valid_states):
            upx, upy = validcoord[xx+1]
            x, y = validcoord[yy+1]
            # 计算上一次在cell 1（_x_1, _y_1），下一步到cell 2 （_x, _y）的概率   
            #check whether coordinate is next to new coordinate
            if((x-1 == upx) and (y == upy)):
                transition_matrix[xx][yy]= Probability(upx, upy, grid_world)
            elif ((x+1 == upx) and (y == upy)):
                transition_matrix[xx][yy]= Probability(upx, upy, grid_world)
            elif ((y-1 == upy) and (x == upx)):
                transition_matrix[xx][yy]= Probability(upx, upy, grid_world)
            elif ((y+1 == upy) and (x == upx)):
                transition_matrix[xx][yy]= Probability(upx, upy, grid_world)
            else:# cell 1 == cell 2，或者 很远不可能一步走到的情况
                transition_matrix[xx][yy]= 0 
            
    return states, initial_probability, transition_matrix, validcoord

def Viterbi(observation_seq, distance_matrix,observations,states,initial_prob,transition_matrix):
    trace_back_states=[]
    steps = 0
    states_max_probability = []
    previous_states = []
    states_sequence = []
    emission_probability = []
    steps = len(observation_seq)
    
    if (steps == 0 or len(initial_prob[0]) == 0 ):
        return states_sequence
               
    for step in range(steps):
        emission_probability = eliminate(observation_seq, step,distance_matrix,observations,states)
        if step == 0:
            states_max_probability = initial_prob * emission_probability            
        else:
            states_max_probability, previous_states = maxp_state(states_max_probability * transition_matrix) 
            states_max_probability = states_max_probability * emission_probability
            trace_back_states.append(previous_states)
    
    states_sequence.extend([maxp(states_max_probability)])
    trace_back_states.reverse()
    for previous_state_list in trace_back_states:
        states_sequence.extend([previous_state_list[states_sequence[-1]]])
    states_sequence.reverse()    
    return states_sequence
    
def eliminate(observation_seq, step,distance_matrix,observations,states):
    state_probability = np.ones((1, len(states)))
    tmp_state_probability = []
    #距离矩阵从4*87*166变成4*87
    for i, oseq in enumerate(observation_seq[step]): 
        tmp_state_probability.append([])
        for j, s in enumerate(states):
            for k, observation in enumerate(observations):
                if(oseq == observation): 
                    tmp_state_probability[i].extend([distance_matrix[i][j][k]]) 
                    break
    for tmp_probability in tmp_state_probability:
        state_probability *= tmp_probability 
    return state_probability

def maxp_state(states_probability):
    max_probability = 0
    max_previous_state = 0
    state_len = len(STATES)
    max_probability_list = []
    max_previous_state_list = []
    for current_state in range(state_len):
        max_probability = 0
        previous_state = 0
        for previous_state in range(state_len):
            probability = states_probability[current_state][previous_state]
            if (probability > max_probability):
                max_probability = probability
                max_previous_state = previous_state
        max_probability_list.extend([max_probability])  
        max_previous_state_list.extend([max_previous_state]) 
    return max_probability_list, max_previous_state_list

def maxp(states_probability):#返回的是最大概率对应的cell的state
    state = 0        
    max_probability = 0
    for st, probability in enumerate(states_probability[0]):
        if (probability > max_probability):
            max_probability = probability
            state = st  
    return state
    
def Find_coo(sequence_list, labels_dictionary):
    sequence_list = sequence_list
    label_sequence = []    
    for sequence in sequence_list:
        label_sequence.extend([labels_dictionary[sequence+1]])    
    return label_sequence

           

GRID_WORLD, TOWER_LOCATION, NOISY_DISTANCES = Load_data('hmm-data.txt')
STATES, INITIAL_PROB, TRANSITION_MATRIX, AvailableCOORDINATION = CPT(GRID_WORLD)
DISTANCE_MATRIX, OBSERVATIONS = Distance(TOWER_LOCATION, GRID_WORLD)
state_sequence = Viterbi(NOISY_DISTANCES,DISTANCE_MATRIX,OBSERVATIONS,STATES,INITIAL_PROB,TRANSITION_MATRIX)
print ('Coordination sequence:', Find_coo(state_sequence, AvailableCOORDINATION))


Coordination sequence: [[5, 3], [6, 3], [7, 3], [7, 2], [7, 1], [7, 2], [7, 1], [6, 1], [5, 1], [4, 1], [3, 1]]
